# Conversión de prescripciones en protocolos clínicos

In [1]:
import aclinprot as acp
import pandas as pd

## Procedimiento
- Exportar la prescripción en ARIA en formato `csv`
- Crear una primera version del protoccolo clínico a partir de la prescripción
- Exportar el archivo de estructuras del paciente
- Cambiar los nombres de las estructuras en la prescripción para hacerlos coincidir con los de contorneo

Crear protocolo clínico a partir de la prescripción

In [12]:
# Nombre del archivo csv con los datos de la prescripción
prescriptionFile = '../../prescripciones/659373.csv'
# Campos del protocolo
ProtocolID='testMama15fx'
TreatmentSite='Breast'
PlanID='MamaDerSIB'

# Primera versión del protocolo clínico a partir de la prescripción
acp.convertPrescriptionIntoClinicalProtocol(prescriptionFile, ProtocolID, TreatmentSite, PlanID)

Nombres de las esructuras dados en el contorneo y en la prescripción

In [3]:
# Nombre del archivo de estructuras
structureFile = '../../DICOM/RS.1.2.246.352.71.4.964456977994.217004.20231212172248.dcm'

# DataFrame para comparar los nombres de las estructuras
contStrdf = pd.DataFrame(acp.readContouringStructureNames(structureFile), columns=['Contouring'])
presStrdf = pd.DataFrame(acp.readClinProtStructureNames('ClinicalProtocol.xml'), columns=['Prescription'])
pd.concat([contStrdf, presStrdf], axis=1)

,Contouring,Prescription
0,BODY,PTV mama dcha
1,Tiroides,PTV boost dcho
2,Pulmón Izqdo,Pulmon derecho
3,Pulmón Dcho,Medula
4,PTV mama dcha,Corazón
5,PTV boost dcho,Tiroides
6,PRV DA,pulmon izquierdo
7,Médula,mama izquierda
8,Mama izqda,NaN
9,Mama dcha,NaN


In [4]:
strNameSuggestions = acp.suggestStrNames(clinprot='ClinicalProtocol.xml',
                                         rsdicom=structureFile)
strNameSuggestions

,Structure,Suggestion
0,PTV mama dcha,PTV mama dcha
1,PTV boost dcho,PTV boost dcho
2,Pulmon derecho,Pulmón Dcho
3,Medula,Médula
4,Corazón,Corazón/pericard
5,Tiroides,Tiroides
6,pulmon izquierdo,Pulmón Izqdo
7,mama izquierda,Mama izqda


Corregir los nombres de las estructuras en la prescripción y recrear el protocolo clínico

In [13]:
# Corregir prescripción
strNamesChanges = strNameSuggestions.copy(deep=True)
strNamesChanges.rename({'Structure' : 'Old', 'Suggestion' : 'New'}, axis=1, inplace=True)
acp.correctStrNames(prescriptionFile, strNamesChanges)

# Recrear protocolo
acp.convertPrescriptionIntoClinicalProtocol(prescriptionFile, ProtocolID, TreatmentSite, PlanID)